In [2]:
import pandas as pd

# 1. test 데이터 불러오기
test_df = pd.read_parquet("../../data/통합_test_데이터.parquet")

In [3]:
from modules.data_loader import map_categorical_columns
from sklearn.preprocessing import LabelEncoder
from modules.feature_selector import selected_cols

# selected_cols + base 컬럼(ID) 결합
final_cols = selected_cols + ["ID"]

# 컬럼 선택해서 복제
df_test_selected = test_df[final_cols].copy()
df_test_encoded = map_categorical_columns(df_test_selected)
for col in df_test_encoded.columns:
    if col != "ID" and df_test_encoded[col].dtype == "object":
        le = LabelEncoder()
        df_test_encoded[col] = le.fit_transform(df_test_encoded[col].astype(str))

[연회비발생카드수_B0M] 인코딩 완료
[한도증액횟수_R12M] 인코딩 완료
[이용금액대] 중간값 인코딩 완료
[할인건수_R3M] 인코딩 완료
[할인건수_B0M] 인코딩 완료
[방문횟수_PC_R6M] 인코딩 완료
[방문횟수_앱_R6M] 인코딩 완료
[방문일수_PC_R6M] 인코딩 완료


In [4]:
from modules.feature_selector import generate_e_features,generate_vif_derived_features
df_test_encoded = generate_e_features(df_test_encoded)
test_df=generate_vif_derived_features(df_test_encoded)

In [65]:
import pandas as pd

# 피처 목록 CSV 로드
feature_list_df = pd.read_csv("../../data/최종_사용_피처목록.csv")

# 피처 이름 리스트 추출
selected_features = feature_list_df["selected_features"].tolist()

In [ ]:
test_df=generate_vif_derived_features(test_df)

In [50]:
# ID와 Segment가 있다면 포함
base_cols = ["ID", "Segment"]  # 또는 "target"으로 바꿔도 됨

# 실제 사용할 전체 피처
use_cols = [col for col in base_cols + selected_features if col in test_df.columns]

# 최종 피처셋 구성
df_selected = test_df[use_cols].copy()

In [51]:
X_test = df_selected.copy()

In [67]:
import xgboost as xgb
import pickle

# 모델과 피처 리스트 함께 로드
with open("./models/final_model.pkl", "rb") as f:
    loaded_model, used_features = pickle.load(f)


[08:26:16] WARNING: D:\bld\xgboost-split_1700181085428\work\src\learner.cc:553: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.



In [72]:
used_features

['잔액_할부_B1M',
 '여유_숙박이용금액',
 '_1순위업종_이용금액',
 '잔액_할부_B0M',
 '최종카드론_대출이율',
 '이용후경과월_할부',
 '회원여부_연체',
 '최대이용금액_일시불_R12M',
 '잔액_현금서비스_B2M',
 '이용카드수_신용',
 '잔액_카드론_B5M',
 '이용금액_할부_무이자_R12M',
 '이용금액_R3M_신용',
 'CA이자율_할인전',
 '방문일수_PC_B0M',
 '방문일수_PC_R6M',
 '평잔_일시불_3M',
 'CA한도금액',
 '이용카드수_신용체크',
 '쇼핑_온라인_이용금액',
 'RV약정청구율',
 '포인트_이용포인트_R3M',
 '이용메뉴건수_ARS_B0M',
 '이용건수_온라인_R6M',
 '정상입금원금_B5M',
 '평잔_일시불_해외_6M',
 '할부금액_무이자_3M_R12M',
 'RP건수_교통_B0M',
 '이용여부_3M_해외겸용_본인',
 '평잔_할부_3M',
 '포인트_마일리지_월적립_R3M',
 '카드이용한도금액',
 'RV일시불이자율_할인전',
 '최종카드론_대출금액',
 '교통_정비이용금액',
 '청구서발송여부_B0',
 '잔액_카드론_B3M',
 '이용카드수_신용_가족',
 '이용금액_일시불_R6M',
 '할부건수_무이자_3M_R12M',
 '변동률_RV일시불평잔',
 '당사멤버쉽_방문월수_R6M',
 '이용후경과월_할부_무이자',
 '이용금액_R3M_체크',
 '잔액_할부_해외_B0M',
 '잔액_할부_B2M',
 '인입횟수_IB_R6M',
 '변동률_할부평잔',
 '인입후경과월_ARS',
 '이용후경과월_부분무이자',
 '잔액_카드론_B0M',
 '카드론이용건수_누적',
 '이용금액_선결제_R6M',
 'RV현금서비스이자율_할인전',
 '청구서발송여부_R3M',
 '포인트_마일리지_건별_R3M',
 '이용건수_신용_R6M',
 '승인거절건수_R3M',
 '연속유실적개월수_기본_24M_카드',
 '포인트_적립포인트_R12M',
 '이용금액_일시불_R3M',
 '증감율_이용금액_일

In [71]:
# test 데이터 컬럼 맞추기
df_test_input = df_test_encoded[used_features]

# DMatrix 생성 후 예측
dtest = xgb.DMatrix(df_test_input)
pred = loaded_model.predict(dtest)

KeyError: "['신용이용건수_증가량', '일시불이용금액_최근비중', '카드론_의존도', '카드론_급증지표', '정상청구_비율', '정상청구_추세', '리볼빙_잔액비율', '방문빈도_앱', '일시불_주사용카드집중도', '이용건수_신판_R12M_minus_이용건수_신용_R12M', '이용건수_신판_R12M_div_이용건수_신용_R12M', '이용건수_일시불_R12M_minus_이용건수_신판_R12M', '이용건수_일시불_R12M_div_이용건수_신판_R12M', '이용건수_일시불_R12M_minus_이용건수_신용_R12M', '이용건수_일시불_R12M_div_이용건수_신용_R12M', '잔액_신판평균한도소진율_r3m_minus_잔액_신판최대한도소진율_r3m', '잔액_신판평균한도소진율_r3m_div_잔액_신판최대한도소진율_r3m', '잔액_카드론_B1M_minus_잔액_카드론_B0M', '잔액_카드론_B1M_div_잔액_카드론_B0M', '잔액_신판ca최대한도소진율_r3m_minus_잔액_신판ca평균한도소진율_r3m', '잔액_신판ca최대한도소진율_r3m_div_잔액_신판ca평균한도소진율_r3m', '포인트_마일리지_건별_R3M_minus_포인트_마일리지_건별_B0M', '포인트_마일리지_건별_R3M_div_포인트_마일리지_건별_B0M', '방문횟수_앱_B0M_minus_방문일수_앱_B0M', '방문횟수_앱_B0M_div_방문일수_앱_B0M', '방문횟수_PC_B0M_minus_방문일수_PC_B0M', '방문횟수_PC_B0M_div_방문일수_PC_B0M', '이용금액_일시불_R3M_minus_이용금액_일시불_B0M', '이용금액_일시불_R3M_div_이용금액_일시불_B0M', '이용금액_일시불_R3M_minus_이용금액_일시불_R6M', '이용금액_일시불_R3M_div_이용금액_일시불_R6M', '이용건수_신용_R6M_minus_이용건수_신용_B0M', '이용건수_신용_R6M_div_이용건수_신용_B0M', '잔액_신판ca최대한도소진율_r6m_minus_잔

In [5]:
file_path = "../../data/통합_train_데이터.parquet"

In [83]:
import pandas as pd
from modules.feature_selector import ab_feat, pca_cols

# ✅ ID, Segment 포함 컬럼 리스트 구성
ab_cols = pca_cols + ab_feat + ["ID", "Segment"]


# ✅ 필요한 컬럼만 로드
ab_df = pd.read_parquet(file_path, columns = ab_cols)
ab_df = ab_df.loc[:, ~ab_df.columns.duplicated()]

In [84]:
from modules.data_loader import map_categorical_columns
from sklearn.preprocessing import LabelEncoder
from modules.feature_selector import selected_cols, generate_vif_derived_features, generate_e_features

# Step 1. ID, Segment 제외한 가공 대상 컬럼만 분리
exclude_cols = ["ID", "Segment"]
target_col = "Segment"
categorical_cols = [col for col in ab_df.columns if ab_df[col].dtype == "object" and col not in exclude_cols]

# Step 2. 복사본 생성
df_processed = ab_df.copy()
df_processed = map_categorical_columns(df_processed)

# Step 3. 이상값 처리 + 범주형 인코딩
for col in categorical_cols:
    df_processed[col] = df_processed[col].replace(['?', '알파벳', '기타'], pd.NA)
    le = LabelEncoder()
    df_processed[col] = le.fit_transform(df_processed[col].astype(str))


[이용금액대] 중간값 인코딩 완료


In [85]:
def make_abcde_stage1_label(x):
    if x == "A":
        return 0  # A
    elif x == "B":
        return 1  # B
    else:
        return 2  # 기타 (C, D, E)
    
df_processed["Stage1_label"] = df_processed["Segment"].apply(make_abcde_stage1_label)


In [86]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import classification_report

X = df_processed.drop(columns=["ID", "Segment", "Stage1_label"])  # 전처리된 피처만
y = df_processed["Stage1_label"]
print("✅ X shape:", X.shape)
print("✅ y 분포:\n", y.value_counts())

✅ X shape: (2400000, 69)
✅ y 분포:
 Stage1_label
2    2398884
0        972
1        144
Name: count, dtype: int64


In [87]:
# X = ab_feat만 추출 (ID, Segment 제외)
X = df_processed[ab_feat].copy()

# y = Segment (라벨 인코딩된 상태 또는 인코딩 필요)
y = df_processed["Stage1_label"]

In [88]:
from sklearn.model_selection import train_test_split

# Stratify를 적용해 클래스 비율을 유지한 채 분리
X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,         # 검증셋 비율 (20%)
    random_state=42,       # 재현성 고정
    stratify=y             # 클래스 비율 유지 (불균형 대비)
)

# 확인
print("✅ 학습셋 크기:", X_train.shape, y_train.shape)
print("✅ 검증셋 크기:", X_val.shape, y_val.shape)
print("✅ 학습셋 클래스 분포:\n", y_train.value_counts(normalize=True))
print("✅ 검증셋 클래스 분포:\n", y_val.value_counts(normalize=True))

✅ 학습셋 크기: (1920000, 30) (1920000,)
✅ 검증셋 크기: (480000, 30) (480000,)
✅ 학습셋 클래스 분포:
 Stage1_label
2    0.999535
0    0.000405
1    0.000060
Name: proportion, dtype: float64
✅ 검증셋 클래스 분포:
 Stage1_label
2    0.999535
0    0.000404
1    0.000060
Name: proportion, dtype: float64


In [89]:
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_sample_weight

# ✅ sample_weight 계산
sample_weights = compute_sample_weight(class_weight='balanced', y=y_train)

# ✅ 모델 정의
model = XGBClassifier(
    objective="multi:softprob",
    num_class=3,
    eval_metric="mlogloss",
    tree_method="gpu_hist",
    predictor="gpu_predictor",
    learning_rate=0.1,
    max_depth=6,
    verbosity=0,
    use_label_encoder=False
)

# ✅ 모델 학습 (with sample_weight)
model.fit(X_train, y_train, sample_weight=sample_weights)

# ✅ 예측
y_pred = model.predict(X_val)

# ✅ 평가
print("✅ Classification Report:")
print(classification_report(y_val, y_pred))

print("✅ Confusion Matrix:")
print(confusion_matrix(y_val, y_pred))


c:\Users\somee\anaconda3\Lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


✅ Classification Report:
              precision    recall  f1-score   support

           0       0.06      0.87      0.11       194
           1       0.10      0.59      0.17        29
           2       1.00      0.99      1.00    479777

    accuracy                           0.99    480000
   macro avg       0.39      0.82      0.42    480000
weighted avg       1.00      0.99      1.00    480000

✅ Confusion Matrix:
[[   168      9     17]
 [     8     17      4]
 [  2714    149 476914]]


In [90]:
# SMOTE 적용 전에 컬럼명 추출해두기
used_features = X.columns.tolist()

In [91]:
from imblearn.over_sampling import SMOTE
from sklearn.impute import SimpleImputer

# 결측치 처리
imputer = SimpleImputer(strategy="mean")
X_train = imputer.fit_transform(X_train)
X_val = imputer.transform(X_val)

# 3-1. SMOTE 적용
smote = SMOTE(random_state=42)
X_train_s, y_train_s = smote.fit_resample(X_train, y_train)

In [92]:
model.fit(X_train_s, y_train_s)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_class=3, num_parallel_tree=None, objective='multi:softprob', ...)

In [81]:
# ✅ 평가
print("✅ Classification Report:")
print(classification_report(y_val, y_pred))

print("✅ Confusion Matrix:")
print(confusion_matrix(y_val, y_pred))

✅ Classification Report:
              precision    recall  f1-score   support

           0       0.06      0.87      0.11       194
           1       0.10      0.59      0.17        29
           2       1.00      0.99      1.00    479777

    accuracy                           0.99    480000
   macro avg       0.39      0.82      0.42    480000
weighted avg       1.00      0.99      1.00    480000

✅ Confusion Matrix:
[[   168      9     17]
 [     8     17      4]
 [  2714    149 476914]]


In [57]:
from modules.model_utils import apply_best_threshold

for cls in [0, 1]:
    print(f"\n📌 클래스 {cls} 기준 Threshold 튜닝:")
    apply_best_threshold(model, X_val, (y_val == cls).astype(int))



📌 클래스 0 기준 Threshold 튜닝:

🌟 최적 Threshold: 0.100 (F1 Score: 0.0643)

📌 클래스 1 기준 Threshold 튜닝:

🌟 최적 Threshold: 0.900 (F1 Score: 0.3409)


In [61]:
# 전체 예측 확률
proba = model.predict_proba(X_val)

# 최적 threshold를 반영한 커스텀 예측
best_thresholds = {0: 0.10, 1: 0.90}  # 클래스 0, 1에 각각 적용
# 커스텀 threshold 적용 함수 호출!
y_pred_custom = custom_threshold(proba, thresh_dict=best_thresholds)



In [62]:
# 평가
from sklearn.metrics import classification_report, confusion_matrix
print("✅ Classification Report (Custom Threshold):")
print(classification_report(y_val, y_pred_custom))

print("✅ Confusion Matrix:")
print(confusion_matrix(y_val, y_pred_custom))

✅ Classification Report (Custom Threshold):
              precision    recall  f1-score   support

           0       0.03      0.95      0.07       194
           1       0.25      0.52      0.34        29
           2       1.00      0.99      0.99    479777

    accuracy                           0.99    480000
   macro avg       0.43      0.82      0.47    480000
weighted avg       1.00      0.99      0.99    480000

✅ Confusion Matrix:
[[   184      3      7]
 [    12     15      2]
 [  5128     41 474608]]


In [93]:
# 모델 학습 후 저장 시 함께 저장
import pickle
import numpy as np

# 모델 저장
with open("./models/final_AB_.pkl", "wb") as f:
    pickle.dump((model, used_features), f)

# 전략 2. ABCD 모델 분류 적용

In [2]:
file_path = "../../data/통합_train_데이터.parquet"

In [10]:
import pandas as pd
from modules.feature_selector import ab_feat, selected_cols

# ✅ ID, Segment 포함 컬럼 리스트 구성
abcd_cols = selected_cols + ["ID", "Segment"]


# ✅ 필요한 컬럼만 로드
abcd_df = pd.read_parquet(file_path, columns = abcd_cols)
abcd_df = abcd_df.loc[:, ~abcd_df.columns.duplicated()]

In [8]:
print(len(abcd_df.columns))
abcd_df = abcd_df[abcd_df['Segment'].isin(['A', 'B', 'C', 'D'])].copy()

80


In [12]:
from modules.data_loader import map_categorical_columns
from sklearn.preprocessing import LabelEncoder
from modules.feature_selector import selected_cols

# Step 1. ID, Segment 제외한 가공 대상 컬럼만 분리
exclude_cols = ["ID", "Segment"]
target_col = "Segment"
categorical_cols = [col for col in abcd_df.columns if abcd_df[col].dtype == "object" and col not in exclude_cols]

# Step 2. 복사본 생성
df_processed = abcd_df.copy()
df_processed = map_categorical_columns(df_processed)

# Step 3. 이상값 처리 + 범주형 인코딩
for col in categorical_cols:
    df_processed[col] = df_processed[col].replace(['?', '알파벳', '기타'], pd.NA)
    le = LabelEncoder()
    df_processed[col] = le.fit_transform(df_processed[col].astype(str))


[이용금액대] 중간값 인코딩 완료


In [19]:
ab_cd_df = df_processed[df_processed['Segment'].isin(['A', 'B', 'C', 'D'])].copy()

In [22]:
from sklearn.discriminant_analysis import StandardScaler
from sklearn.model_selection import train_test_split
from modules.preprocess_utils import get_clean_numeric_columns
from xgboost import XGBClassifier

X = ab_cd_df[selected_cols].copy()
y = ab_cd_df["Segment"]

X = X.loc[:, ~X.columns.duplicated()]

raw_num_cols = X.columns.tolist()  # 전체 대상 중에서 검사
num_cols = get_clean_numeric_columns(X, raw_num_cols)

scaler = StandardScaler()
X_scaled_num = pd.DataFrame(
    scaler.fit_transform(X[num_cols]),
    columns=num_cols,
    index=X.index  # 인덱스 유지
)

# 스케일링 안 한 범주형/문자형 컬럼은 그대로 유지
other_cols = [col for col in X.columns if col not in num_cols]
X_scaled = pd.concat([X_scaled_num, X[other_cols]], axis=1)

# 6. 타겟 라벨 인코딩
le_y = LabelEncoder()
y_encoded = le_y.fit_transform(y)

# 7. train-validation 분할
X_train, X_val, y_train, y_val = train_test_split(
    X_scaled, y_encoded, test_size=0.2, stratify=y_encoded, random_state=42
)

# 8. XGBoost 모델 선언 (GPU 가속)
xgb_model = XGBClassifier(
    tree_method='gpu_hist',
    predictor='gpu_predictor',
    n_estimators=300,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    use_label_encoder=False,
    eval_metric='mlogloss',
    random_state=42
)

# 9. 학습
xgb_model.fit(X_train, y_train)

c:\Users\somee\anaconda3\Lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.05, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=300, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob',
              predictor='gpu_predictor', ...)

In [23]:
from sklearn.metrics import classification_report, confusion_matrix
y_pred = xgb_model.predict(X_val)
print(classification_report(y_val, y_pred))


              precision    recall  f1-score   support

           0       0.90      0.27      0.42       194
           1       0.90      0.31      0.46        29
           2       0.78      0.59      0.67     25518
           3       0.86      0.94      0.90     69849

    accuracy                           0.85     95590
   macro avg       0.86      0.53      0.61     95590
weighted avg       0.84      0.85      0.84     95590



In [24]:
from modules.model_utils import apply_smote
X_smote, y_smote = apply_smote(X_train, y_train, random_state=42)

✅ SMOTE 적용 완료
  → Before: 382358 samples
  → After:  1117572 samples (class balanced)


In [25]:
xgb_model.fit(X_smote, y_smote)
from sklearn.metrics import classification_report, confusion_matrix
y_pred = xgb_model.predict(X_val)
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.24      0.88      0.37       194
           1       0.48      0.79      0.60        29
           2       0.66      0.69      0.68     25518
           3       0.89      0.87      0.88     69849

    accuracy                           0.82     95590
   macro avg       0.57      0.81      0.63     95590
weighted avg       0.83      0.82      0.83     95590



In [27]:
# 모델 학습 후 저장 시 함께 저장
import pickle

# 학습 피처 리스트
used_features = X_train.columns.tolist()

# 모델 저장
with open("./models/final_AtoD_.pkl", "wb") as f:
    pickle.dump((xgb_model, used_features), f)

In [34]:
from sklearn.metrics import f1_score
from modules.model_utils import custom_threshold    
import numpy as np

# 예측 확률
proba = xgb_model.predict_proba(X_val)

# 결과 저장용
best_thresholds = {}
threshold_range = np.linspace(0.1, 0.9, 20)

# 클래스별로 best threshold 찾기 (0, 1 클래스만)
for cls in [0, 1, 2 , 3 ]:
    best_f1 = 0
    best_thresh = 0.5
    
    for t in threshold_range:
        y_pred = custom_threshold(proba, thresh_dict={cls: t}, default_class=2)
        f1 = f1_score(y_val, y_pred, average="macro")
        
        if f1 > best_f1:
            best_f1 = f1
            best_thresh = t
    
    best_thresholds[cls] = best_thresh
    print(f"📌 클래스 {cls} → Best Threshold: {best_thresh:.2f} (F1: {best_f1:.4f})")

print("\n🎯 전체 best threshold 조합:", best_thresholds)


📌 클래스 0 → Best Threshold: 0.86 (F1: 0.2441)
📌 클래스 1 → Best Threshold: 0.82 (F1: 0.2813)
📌 클래스 2 → Best Threshold: 0.10 (F1: 0.1054)
📌 클래스 3 → Best Threshold: 0.48 (F1: 0.3930)

🎯 전체 best threshold 조합: {0: 0.8578947368421053, 1: 0.8157894736842105, 2: 0.1, 3: 0.4789473684210527}


In [40]:
from sklearn.metrics import classification_report, confusion_matrix

# best threshold 기반 예측
y_pred_final = custom_threshold(proba, best_thresholds, default_class=1)

# 전체 평가
print("\n✅ 📊 Classification Report (Best Thresholds 적용):")
print(classification_report(y_val, y_pred_final))

print("✅ Confusion Matrix:")
print(confusion_matrix(y_val, y_pred_final))


✅ 📊 Classification Report (Best Thresholds 적용):
              precision    recall  f1-score   support

           0       0.54      0.57      0.56       194
           1       0.58      0.72      0.65        29
           2       0.67      0.71      0.69     25518
           3       0.89      0.87      0.88     69849

    accuracy                           0.83     95590
   macro avg       0.67      0.72      0.69     95590
weighted avg       0.83      0.83      0.83     95590

✅ Confusion Matrix:
[[  110     2    82     0]
 [    0    21     8     0]
 [   82     9 18157  7270]
 [   10     4  8998 60837]]
